In [3]:
import pandas as pd
import numpy as np


In [6]:
df = pd.read_feather('../data/train_dataset.fth')
df.head()


,index,id,cell_id,cell_type,source,rank,pct_rank,ancestor_id,parent_id
0,0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many he...,0,0.000000,945aea18,None
1,1,00001756c60be8,448eb224,markdown,**Импортируем необходимые для работы функции и...,1,0.017241,945aea18,None
2,2,00001756c60be8,2a9e43d6,code,import numpy as np\nimport pandas as pd\nimpor...,2,0.034483,945aea18,None
3,3,00001756c60be8,7e2f170a,markdown,**Подключаем предупреждения**,3,0.051724,945aea18,None
4,4,00001756c60be8,038b763d,code,import warnings\nwarnings.filterwarnings('igno...,4,0.068966,945aea18,None


In [26]:
plot_functions = {
    'histplot', 'kdeplot', 'displot', 
    'displot', 'scatterplot', 'relplot', 
    'jointplot', 'plot', 'show',
    'pairplot', 'countplot', 'jointplot'
    'relplot', 'lmplot', 'catplot',
    'scatter', 'hist'
}

In [140]:
from tqdm import tqdm
import tokenize
import io


def get_functions_and_variables(source):
    try:
        code_text = tokenize.generate_tokens(io.StringIO(source).readline)
        fv_list = [tok.string for tok in code_text if tok.type==1]
    except (tokenize.TokenError, IndentationError) as e:
        return [], []
        
    fv_plot = [s for s in fv_list if any([s in p for p in plot_functions])]
    return fv_list, fv_plot

    
def generate_features(df):
    features = dict()
    
    cell_counts = df.groupby('id').cell_type.value_counts().unstack()
    cell_counts['ratio'] = cell_counts['markdown'] / cell_counts['code']
    cell_counts['total'] = (cell_counts['markdown'] + cell_counts['code']).astype(int)
    mean_ratios_by_count = cell_counts.groupby('total').ratio.mean()    
    
    for idx, sub_df in tqdm(df.groupby("id")):
        features[idx] = dict()
        total_md = cell_counts.loc[idx].markdown
        total_code = cell_counts.loc[idx].code

        code_sub_df = sub_df[sub_df.cell_type == "code"]
        source_code = '\n'.join(code_sub_df.source)
        funcs_and_vars, plot_funcs = get_functions_and_variables(source_code)
        defined_functions = re.findall('def (.*)\(', source_code)
        
        features[idx] = {
            "total_code": total_code, 
            "total_md": total_md,
            
            "defined_functions": " ".join(defined_functions),
            "normalized_defined_functions": len(plot_funcs) / total_code,
            "normalized_plot_functions": len(defined_functions) / total_code,
            
            "mean_ratio_in_same_notebooks": mean_ratios_by_count.loc[total_md + total_code],
            "normalized_sloc": len(source_code.splitlines()) / total_code,
        }
        
    return features


In [143]:
ft = generate_features(df.iloc[:1000])


100%|██████████| 18/18 [00:00<00:00, 106.93it/s]


In [144]:
ft

{'00001756c60be8': {'total_code': 30.0,
  'total_md': 28.0,
  'defined_functions': 'evaluate_preds __init__ fit transform features',
  'normalized_defined_functions': 0.5333333333333333,
  'normalized_plot_functions': 0.16666666666666666,
  'mean_ratio_in_same_notebooks': 0.9333333333333333,
  'normalized_sloc': 9.0},
 '00015c83e2717b': {'total_code': 72.0,
  'total_md': 21.0,
  'defined_functions': 'mask2rgba name_and_mask show_mask_image mask2rle rle2mask vflip hflip random_flip transpose rot90 rotate shift_scale_rotate center_crop clip clipped wrapped_function shift_hsv shift_rgb clahe(img, clipLimit=2.0, tileGridSize= blur median_blur motion_blur random_polosa distort1 distort2 elastic_transform_fast remap_color get_lut invert channel_shuffle gauss_noise salt_pepper_noise poisson_noise speckle_noise random_brightness random_contrast to_three_channel_gray to_gray add_channel fix_mask img_to_tensor mask_to_tensor _go_ben __init__ forward',
  'normalized_defined_functions': 1.98611111

In [138]:
df['kek'] = df.groupby('id')['index'].transform('count')


In [139]:
df

,index,id,cell_id,cell_type,source,rank,pct_rank,ancestor_id,parent_id,kek
0,0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many he...,0,0.000000,945aea18,None,58
1,1,00001756c60be8,448eb224,markdown,**Импортируем необходимые для работы функции и...,1,0.017241,945aea18,None,58
2,2,00001756c60be8,2a9e43d6,code,import numpy as np\nimport pandas as pd\nimpor...,2,0.034483,945aea18,None,58
3,3,00001756c60be8,7e2f170a,markdown,**Подключаем предупреждения**,3,0.051724,945aea18,None,58
4,4,00001756c60be8,038b763d,code,import warnings\nwarnings.filterwarnings('igno...,4,0.068966,945aea18,None,58
...,...,...,...,...,...,...,...,...,...,...
6370641,6370641,fffe1d764579d5,5369934b,markdown,* We have achieved 80.8% accuracy in predictin...,67,0.930556,3c40bfa6,None,72
6370642,6370642,fffe1d764579d5,ac9db030,code,from sklearn.tree import DecisionTreeRegressor...,68,0.944444,3c40bfa6,None,72
6370643,6370643,fffe1d764579d5,a8ffc8b4,markdown,* We have achieved 75.2% accuracy in predictin...,69,0.958333,3c40bfa6,None,72
6370644,6370644,fffe1d764579d5,70455170,code,from sklearn.ensemble import RandomForestRegre...,70,0.972222,3c40bfa6,None,72


In [150]:
df.iloc[:1000].merge(pd.DataFrame.from_dict(ft, orient='index'), left_on='id', right_index=True)

,index,id,cell_id,cell_type,source,rank,pct_rank,ancestor_id,parent_id,kek,total_code,total_md,defined_functions,normalized_defined_functions,normalized_plot_functions,mean_ratio_in_same_notebooks,normalized_sloc
0,0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many he...,0,0.000000,945aea18,None,58,30.0,28.0,evaluate_preds __init__ fit transform features,0.533333,0.166667,0.933333,9.0
1,1,00001756c60be8,448eb224,markdown,**Импортируем необходимые для работы функции и...,1,0.017241,945aea18,None,58,30.0,28.0,evaluate_preds __init__ fit transform features,0.533333,0.166667,0.933333,9.0
2,2,00001756c60be8,2a9e43d6,code,import numpy as np\nimport pandas as pd\nimpor...,2,0.034483,945aea18,None,58,30.0,28.0,evaluate_preds __init__ fit transform features,0.533333,0.166667,0.933333,9.0
3,3,00001756c60be8,7e2f170a,markdown,**Подключаем предупреждения**,3,0.051724,945aea18,None,58,30.0,28.0,evaluate_preds __init__ fit transform features,0.533333,0.166667,0.933333,9.0
4,4,00001756c60be8,038b763d,code,import warnings\nwarnings.filterwarnings('igno...,4,0.068966,945aea18,None,58,30.0,28.0,evaluate_preds __init__ fit transform features,0.533333,0.166667,0.933333,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,000a2f5243e1ca,75dc5015,code,corpus,18,0.400000,ae9b5889,None,45,20.0,3.0,,0.550000,0.000000,0.725000,1.9
996,996,000a2f5243e1ca,39ceb8e0,markdown,WORD EMBEDDING REPRESENTATION,19,0.422222,ae9b5889,None,45,20.0,3.0,,0.550000,0.000000,0.725000,1.9
997,997,000a2f5243e1ca,62c1199d,code,import tensorflow as tf\nfrom tensorflow.keras...,20,0.444444,ae9b5889,None,45,20.0,3.0,,0.550000,0.000000,0.725000,1.9
998,998,000a2f5243e1ca,7b965d59,code,"rep=[one_hot(i,voc_size) for i in corpus]",21,0.466667,ae9b5889,None,45,20.0,3.0,,0.550000,0.000000,0.725000,1.9


In [151]:
ft['000a2f5243e1ca']

{'total_code': 20.0,
 'total_md': 3.0,
 'defined_functions': '',
 'normalized_defined_functions': 0.55,
 'normalized_plot_functions': 0.0,
 'mean_ratio_in_same_notebooks': 0.725,
 'normalized_sloc': 1.9}